In [1]:

"""
@author: pickle0412
"""

import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torchvision
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
%matplotlib inline
from numpy import moveaxis
from skimage import io, transform
from torch.autograd import Variable
from PIL import Image
import pickle
import os
import glob
import pandas as pd
import random
import pickle
import cv2
import plotly
# import bokeh
from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import warnings
warnings.filterwarnings("ignore")

plt.ion()

In [2]:
NUM_CLASSES=10
CLASS_LABELS=['c' + str(i) for i in range(10)]
CLASSES_LIST=['Drive Safe', 'Text Right', 'Talk Right', 'Text Left', 'Talk Left',
              'Adjust Radio', 'Drink', 'Reach Behind', 'Hair & Makeup', 'Talk Passenger']

In [13]:
def load_train_dataset():
    data_path = '../AUCdata/v2_cam1_cam2_ split_by_driver/Camera 1/train/'
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=torchvision.transforms.Compose([
        transforms.Resize(size=(224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    )
    return train_dataset

def load_test_dataset():
    data_path = '../AUCdata/v2_cam1_cam2_ split_by_driver/Camera 1/test/'
    test_dataset = torchvision.datasets.ImageFolder(
        root = data_path,
        transform = transforms.Compose([
            transforms.Resize(size=(224,224)),
            transforms.ToTensor()
        ])
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size = 16,
        num_workers = 4,
        shuffle = False
    )
    return test_loader

In [14]:
dataset = load_train_dataset()
batch_size = 16
validation_split = .2
shuffle_dataset = True
random_seed= 42

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler, num_workers=4)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler, num_workers=4)

testLoader = load_test_dataset()

In [22]:
vgg = models.vgg16(pretrained=True)
vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [15]:
class VGG_CVPR_original_model(nn.Module):
    def __init__(self):
        super(VGG_CVPR_original_model, self).__init__()
        
        self.block = nn.Sequential(
                      nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(64),
                      nn.LeakyReLU(),
                      nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(64),
                      nn.LeakyReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                      nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(128),
                      nn.LeakyReLU(),
                      nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(128),
                      nn.LeakyReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                      nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(256),
                      nn.LeakyReLU(),
                      nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(256),
                      nn.LeakyReLU(),
                      nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(256),
                      nn.LeakyReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                      nn.Dropout(p=0.1),
                      nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(512),
                      nn.LeakyReLU(),
                      nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(512),
                      nn.LeakyReLU(),
                      nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(512),
                      nn.LeakyReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                      nn.Dropout(p=0.2),
                      nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(512),
                      nn.LeakyReLU(),
                      nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(512),
                      nn.LeakyReLU(),
                      nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(512),
                      nn.LeakyReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
                      nn.Dropout(p=0.3),
#                       nn.AdaptiveAvgPool2d(output_size=(7, 7)),
                      nn.Conv2d(512, 512, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(512),
                      nn.LeakyReLU(),
                      nn.Dropout(p=0.4),
                      nn.Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(512),
                      nn.LeakyReLU(),
                      nn.Dropout(p=0.5),
                      nn.Conv2d(512, 10, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1)),
                      nn.BatchNorm2d(10),
                      nn.LeakyReLU()
        )
        
    def forward(self, x):
                                
        out = self.block(x)
        
        return out

In [16]:
model = VGG_CVPR_original_model()

In [17]:
if torch.cuda.is_available():
    model.cuda()

In [18]:
criterion = nn.CrossEntropyLoss()

In [19]:
learning_rate = 0.0001

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, weight_decay=1e-6, momentum=0.9)

In [20]:
def train(model,dataloaderList,epochs, device = 'cpu',saveName=None):
    
    trainloader=dataloaderList[0]
    valloader=dataloaderList[1]

    valAccList=[]
    
    print('Start Training...')
    for epoch in range(epochs):
        running_loss=0
        
        for i_batch, (images, labels) in enumerate(trainloader):
            
            images=Variable(images,requires_grad=True).to(device)
            labels=Variable(labels).to(device)
            
            optimizer.zero_grad()
            
            outputs=model(images)

            loss=criterion(outputs,labels)
            
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            
            if i_batch%100 == 99:
                print('[%2d,%5d] Training Loss: %.3f'  % 
                      (epoch+1,i_batch+1,running_loss/100))
                running_loss = 0

               
        print('Validating the model')
        
        with torch.no_grad():
            valPreds=[]
            valLabels=[]
            for i_batch, (images, labels) in enumerate(valloader):
                
                images= images.to(device)
                labels= labels.numpy()
                
                preds=model(images)
                preds=preds.cpu().numpy()
                
                pred_labels=np.argmax(preds,axis=-1)
                valPreds.append(pred_labels)
                valLabels.append(labels)
            
            valPreds=np.concatenate(valPreds)
            valLabels=np.concatenate(valLabels)
            size=valPreds.shape[0]
            running_corrects=np.sum(valPreds==valLabels)
        
            acc=float(running_corrects)/size
            valAccList.append(acc)
            print('Accuracy after epoch %2d is %.3f' % (epoch+1,acc))
            
            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, '../Model_weights/'+saveName+'.pt')
    
    print("Finished Training")
    print("Saving Model")
#     torch.save(model.state_dict(),'Model_weights/'+saveName+'.pt')
    
    return model,valAccList
    

In [21]:
custom_model_1, valAccList_1 = train(model, [train_loader, validation_loader], 50, device, 'cvpr_regularized_model')

Start Training...


RuntimeError: 1only batches of spatial targets supported (non-empty 3D tensors) but got targets of size: : [16]

In [ ]:
with open('../classification_models/vgg_cvpr_regularized_model_val_acc_list', 'wb') as f:
    pickle.dump(valAccList_1, f)

In [ ]:
model = VGG_CVPR_original_model
checkpoint = torch.load('Model_weights/cvpr_regularized_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [ ]:
with torch.no_grad():
            valPreds=[]
            valLabels=[]
            valAccList = []
            for i_batch, (images, labels) in enumerate(validation_loader):
                
                images= images.to(device)
                labels= labels.numpy()
                
                preds=model(images)
                preds=preds.cpu().numpy()
                
                pred_labels=np.argmax(preds,axis=-1)
                valPreds.append(pred_labels)
                valLabels.append(labels)
            
            valPreds=np.concatenate(valPreds)
            valLabels=np.concatenate(valLabels)
            size=valPreds.shape[0]
            running_corrects=np.sum(valPreds==valLabels)
        
            acc=float(running_corrects)/size
            valAccList.append(acc)
            print('Accuracy after 15 epochs is %.3f' % (acc))